In [1]:
import pickle
import pandas as pd
from datetime import datetime
import re
import time

class CustomerDataExtractor:
    """
    Loads nested customer orders data and VIP list; transforms into a flat DataFrame.

    Required Output Columns:
      - customer_id: int
      - customer_name: str
      - registration_date: datetime64[ns]
      - is_vip: bool
      - order_id: int
      - order_date: datetime64[ns]
      - product_id: int
      - product_name: str
      - category: str
      - unit_price: float
      - item_quantity: int
      - total_item_price: float
      - total_order_value_percentage: float

    The resulting DataFrame is sorted by customer_id, order_id, product_id.

    Usage:
      extractor = CustomerDataExtractor('customer_orders.pkl', 'vip_customers.txt')
      extractor.load_data()
      df = extractor.transform()
      df.to_csv('customer_orders_flat.csv', index=False)
    """

    CATEGORY_MAP = {
        1: 'Electronics',
        2: 'Apparel',
        3: 'Books',
        4: 'Home Goods'
    }

    def __init__(self, orders_file='customer_orders.pkl', vip_file='vip_customers.txt') -> None:
        self.orders_file = orders_file
        self.vip_file = vip_file
        self.customers = []
        self.vip_ids = set()

    def load_data(self) -> None:
        with open(self.orders_file, 'rb') as f:
            self.customers = pickle.load(f)

        with open(self.vip_file, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    try:
                        self.vip_ids.add(int(line))
                    except ValueError:
                        continue
                        
    @staticmethod
    def _clean_price(price) -> float:
        if isinstance(price, str):
            price = re.sub(r'[^0-9.]', '', price)
        try:
            return float(price)
        except:
            return 0.0
        
    def transform(self) -> pd.DataFrame:
        rows = []
        for cust in self.customers:
            cust_id = cust.get('id', -1)
            if cust_id is None:
                continue
            cust_id = int(cust_id)
            
            cust_name = cust.get('name', '').strip()
            
            reg_date_raw = cust.get('registration_date')
            if not reg_date_raw:
                continue
            reg_date = pd.to_datetime(reg_date_raw, errors='coerce')
            if pd.isna(reg_date):
                continue

            is_vip = cust_id in self.vip_ids

            for order in cust.get('orders', []):
                raw_oid = order.get('order_id')
                if raw_oid is None:
                    continue
                try:
                    order_id = int(raw_oid)
                except (ValueError, TypeError):
                    continue

                raw_odate = order.get('order_date')
                if not raw_odate:
                    continue
                order_date = pd.to_datetime(raw_odate, errors='coerce')
                if pd.isna(order_date):
                    # Skip invalid dates
                    continue

                for item in order.get('items', []):
                    raw_pid = item.get('item_id')
                    if raw_pid is None:
                        continue
                    try:
                        product_id = int(raw_pid)
                    except (ValueError, TypeError):
                        continue

                    product_name = item.get('product_name', "")
                    if (
                        product_name is None                       
                        or (isinstance(product_name, float) and pd.isna(product_name))
                        or (isinstance(product_name, str) and not product_name.strip())
                    ):
                        product_name = f"Item {product_id} for Order {order_id}"
    
                    raw_cat = item.get('category')
                    category = self.CATEGORY_MAP.get(raw_cat, 'Misc')
                    
    
                    unit_price = CustomerDataExtractor._clean_price(item.get('price', '0.0'))
                    
                    try:
                        qty = int(item.get('quantity', 0))
                    except (ValueError, TypeError):
                        qty = 0
                    if qty == 0:
                        continue
                        
                    # why i wrote like this, if someone wants to include qty 0, just comment out the block

                    total_price = unit_price * qty

                    rows.append({
                        'customer_id': cust_id,
                        'customer_name': cust_name,
                        'registration_date': reg_date,
                        'is_vip': is_vip,
                        'order_id': order_id,
                        'order_date': order_date,
                        'product_id': product_id,
                        'product_name': product_name,
                        'category': category,
                        'unit_price': unit_price,
                        'item_quantity': qty,
                        'total_item_price': total_price
                    })

        if not rows:
            columns = [
                'customer_id','customer_name','registration_date','is_vip',
                'order_id','order_date','product_id','product_name','category',
                'unit_price','item_quantity','total_item_price','total_order_value_percentage'
            ]
            return pd.DataFrame(columns=columns)

        df = pd.DataFrame(rows)

        order_totals = df.groupby(['customer_id', 'order_id'])['total_item_price'].transform('sum')
        df['total_order_value_percentage'] = (df['total_item_price'] / order_totals) * 100.0
        
        str_cols = ['customer_name', 'product_name', 'category']
        df[str_cols] = df[str_cols].astype('string') 
        
        df = df.astype({
            'customer_id': 'int64',
            'customer_name': 'string',
            'registration_date': 'datetime64[ns]',
            'is_vip': 'boolean',
            'order_id': 'int64',
            'order_date': 'datetime64[ns]',
            'product_id': 'int64',
            'product_name': 'string',
            'category': 'string',
            'unit_price': 'float64',
            'item_quantity': 'int64',
            'total_item_price': 'float64',
            'total_order_value_percentage': 'float64'
        })

        # Sort
        df = df.sort_values(['customer_id', 'order_id', 'product_id']).reset_index(drop=True)
        return df


if __name__ == "__main__":
    start = time.time()
    extractor = CustomerDataExtractor()
    extractor.load_data()
    df_final = extractor.transform()
    
    out = "customer_orders_flat.csv"
    df_final.to_csv(out, index=False)
    print(f"✓ Saved {len(df_final):,} rows → {out}")
    end = time.time()
    print(f"Time: {end - start}")


✓ Saved 141 rows → customer_orders_flat.csv
Time: 0.07442569732666016


In [2]:
for column in df_final.columns:
    print(f"{column} {df_final[str(column)].dtype} ")

customer_id int64 
customer_name string 
registration_date datetime64[ns] 
is_vip boolean 
order_id int64 
order_date datetime64[ns] 
product_id int64 
product_name string 
category string 
unit_price float64 
item_quantity int64 
total_item_price float64 
total_order_value_percentage float64 
